# Single Node + MongoDB + [DECaLS](https://www.legacysurvey.org/decamls/)

### 이 페이지에서는 로컬 머신 즉 Single Node에서 Python을 이용해 MongoDB에 DECam Legacy Survey 카탈로그를 업로드하고 주어진 좌표와 반경내에서 검색하는데 걸리는 시간을 측정해보도록 하겠습니다.

### 다음 코드를 돌린 컴퓨터의 스펙은 다음과 같습니다.
<img src="my_mac_spec.png" style="width:300px;height:150px" align="left" />

In [72]:
import sys

sys.version

'3.7.6 (default, Jan  8 2020, 13:42:34) \n[Clang 4.0.1 (tags/RELEASE_401/final)]'

* 여기서 사용하는 파이썬의 버젼은 위와 같습니다.

In [9]:
from astropy.table import Table
import json
import time
import os

def table_single_row_to_dict(table):
    """Convert Astropy Table to Python dict.

    Numpy arrays are converted to lists, so that
    the output is JSON serialisable.

    Can work with multi-dimensional array columns,
    by representing them as list of list.
    """
    total_data = {}
    for name in table.colnames:
        if isinstance(table[name], str):
            total_data[name] = table[name]
        else:
            total_data[name] = table[name].tolist()
    return total_data

* Astropy Table 포멧을 Python의 Dictionary 포멧으로 변환하는 함수를 정의합니다.

In [3]:
import pymongo

client = pymongo.MongoClient()  # default connection (ie, local)

db_name = 'legacy-144-bricks-around-A2670'
db = client[db_name]  # database
brick = db.brick  # collection; can also call as db['dwarfs']
brick.drop()  # drop collection, if needed

* 위를 실행하기에 앞서 [MongoDB community edition](https://docs.mongodb.com/manual/installation/#mongodb-community-edition-installation-tutorials)을 인스톨합니다. 
* 'legacy-144-bricks-around-A2670'이라는 db를 만들었습니다.

In [10]:
%%time
cat_dir = '/Users/duhokim/work/abell/cat/legacy/a2670/'
for file in os.listdir(cat_dir):
    if file.endswith('.fits'):
        cat = Table.read(cat_dir + file)
        for i in range(0, len(cat)):
            json_data = json.loads(json.dumps(table_single_row_to_dict(cat[i])))
            result = brick.insert_one(json_data)

CPU times: user 9min 45s, sys: 8.07 s, total: 9min 53s
Wall time: 11min 21s


* Astropy Table 포멧으로 읽은 카탈로그를 Python Dictionary로 변환한 뒤 다시 JSON 포멧으로 변환해 db로 업로드합니다.
* 144개의 brick (~3deg^2)을 업로드 하는 데 약 11분 21초가 걸렸습니다.

In [6]:
# Quick check to confirm load
cursor = brick.find({'objid': 1000})

# for doc in cursor:
#     print(doc)

CPU times: user 30 µs, sys: 0 ns, total: 30 µs
Wall time: 33.1 µs
